<a href="https://colab.research.google.com/github/ZeeeeCS/MarketCanadaStock/blob/main/Marketanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.templates.default = "plotly_white"

In [29]:
! pip install -q opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/atharvaarya25/financials')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: amscientist 
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/atharvaarya25/financials


100%|██████████| 21.5k/21.5k [00:00<00:00, 51.3MB/s]

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
data=pd.read_csv("/content/financials/Financials.csv")

In [33]:
data=data[data['Segment']=='Government']

In [35]:
data=data[data['Country']=='Canada']



In [36]:
data.columns

Index(['Segment', 'Country', ' Product ', ' Discount Band ', ' Units Sold ',
       ' Manufacturing Price ', ' Sale Price ', ' Gross Sales ', ' Discounts ',
       '  Sales ', ' COGS ', ' Profit ', 'Date', 'Month Number',
       ' Month Name ', 'Year'],
      dtype='object')

In [37]:
data[' Product '].unique()

array([' Carretera ', ' Paseo ', ' VTT ', ' Montana ', ' Velo ',
       ' Amarilla '], dtype=object)

In [38]:
data['  Sales ']=data['  Sales '].str.replace('$','').str.replace(',','').astype(float)

In [39]:
data[' Profit ']=data[' Profit '].str.replace('$','').str.replace(',','').astype(float)

In [40]:
sales_by_product=data.groupby(' Product ')['  Sales '].sum()

In [41]:
sales_by_product.reset_index()

,Product,Sales
0,Amarilla,2579791.79
1,Carretera,1332156.65
2,Montana,829859.91
3,Paseo,3956873.45
4,VTT,667982.25
5,Velo,1374572.47


In [42]:
Profit=data.groupby('Year')[' Profit '].sum()

In [43]:
Profit

,Profit
Year,
2013,606726.12
2014,1651745.40


In [44]:
data[' Gross Sales ']=data[' Gross Sales '].str.replace('$','').str.replace(',','').astype(float)

In [45]:
gross_profit=data.groupby('Year')[' Gross Sales '].sum()

In [46]:
gross_profit

,Gross Sales
Year,
2013,2589198.0
2014,9056159.0


In [47]:
data[' Discounts ']=data[' Discounts '].str.replace('$','').str.replace(',','').str.replace('-','').str.replace('    ','0').astype(float)

In [48]:
services=data.groupby('Year')[' Discounts '].sum()
services

,Discounts
Year,
2013,110771.88
2014,793348.60


In [49]:
data[' COGS ']=data[' COGS '].str.replace('$','').str.replace(',','').astype(float)

In [50]:
cost_of_revenue=data.groupby('Year')[' COGS '].sum()

In [51]:
product_cost=cost_of_revenue.subtract(services)



In [52]:
services

,Discounts
Year,
2013,110771.88
2014,793348.60


In [53]:
revenue=gross_profit + product_cost

In [54]:
r_and_d = 1200
sg_a = 1500
taxes = 1800
operating_expences=(r_and_d + sg_a)
operating_profit = gross_profit - (r_and_d + sg_a)
net_profit = operating_profit - taxes

In [55]:
labels = [
    "Carretera", "Paseo", "VTT", "Montana", "Velo", "Amarilla","Products",
    "Revenue", "Cost of revenue","services cost","Products Cost", "Gross Profit","Operateing Expenses","R&D", "SG&A", "Taxes","Operating Profit", "Net Profit"
]

In [56]:
source = [0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 8, 7, 11, 12, 12, 16, 11, 16]
target = [6, 6, 6, 6, 6, 6, 7, None, 8, 9, 10, 11, 12, 12, 12, 12, 16, None]
value = [sales_by_product[0],sales_by_product[1],sales_by_product[2],sales_by_product[3],sales_by_product[4],sales_by_product[5],revenue[2014],revenue[2014],cost_of_revenue[2014],services[2014],product_cost[2014],gross_profit[2014],operating_expences,r_and_d,sg_a,taxes,operating_profit,net_profit[2014]]

In [57]:
import plotly.graph_objects as go

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black"),
        label=labels # Add labels to nodes
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
))
fig.update_layout(title_text="Canada Stock", font_size=10)
fig.show()
